This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/trajopt.html).  I recommend having both windows open, side-by-side!

# Trajectory optimization for the double integrator

In [ ]:
import numpy as np
import pydot
from IPython.display import SVG, display
from matplotlib import pyplot as plt
from pydrake.all import (
    Binding,
    DirectCollocation,
    DirectTranscription,
    GraphOfConvexSets,
    GraphOfConvexSetsOptions,
    HPolyhedron,
    LinearConstraint,
    LinearSystem,
    MathematicalProgram,
    Point,
    Solve,
    eq,
)

# Direct Transcription (using Mathematical Program directly)

In [ ]:
def double_integrator():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")

    def solve_for_fixed_horizon(N):
        prog = MathematicalProgram()

        # Create decision variables
        u = prog.NewContinuousVariables(1, N - 1, "u")
        x = prog.NewContinuousVariables(2, N, "x")

        # Add constraints
        x0 = [-2, 0]
        prog.AddBoundingBoxConstraint(x0, x0, x[:, 0])
        for n in range(N - 1):
            # Will eventually be prog.AddConstraint(x[:,n+1] == A@x[:,n] + B@u[:,n])
            # See drake issues 12841 and 8315
            prog.AddConstraint(eq(x[:, n + 1], A.dot(x[:, n]) + B.dot(u[:, n])))
            prog.AddBoundingBoxConstraint(-1, 1, u[:, n])
            prog.AddQuadraticCost(u[0, n] ** 2, True)
        xf = [0, 0]
        prog.AddBoundingBoxConstraint(xf, xf, x[:, N - 1])

        result = Solve(prog)
        return result, prog, x, u

    # TODO(russt): Do a line search here (I've done it manually).
    N = 284
    # True solution: min time(q=-2, qdot=0) = 2√2 = 2.828 seconds.
    result, prog, x, u = solve_for_fixed_horizon(N)
    assert result.is_success(), "Optimization failed"

    u_sol = result.GetSolution(u)
    x_sol = result.GetSolution(x)

    fig, ax = plt.subplots(2, 1)
    ax[0].plot(x_sol[0, :], x_sol[1, :], "-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    ax[1].plot(np.arange(0, N - 1) * dt, u_sol.T, "-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


double_integrator()

## DirectTranscription (using the DirectTranscription class)

Because this pattern of making decision variables that are indexed over time, adding the dynamic constraints, defining the running cost and constraints, is so common, we have wrappers in drake on top of `MathematicalProgram` which handle these details for you.

The optimization below is identical to the example above, but using this helper class.

In [ ]:
def dirtran_example():
    # Discrete-time approximation of the double integrator.
    dt = 0.01
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")
    C = np.eye(2)
    D = np.zeros((2, 1))
    sys = LinearSystem(A, B, C, D, dt)

    N = 284
    x0 = [-2, 0]
    xf = [0, 0]

    dirtran = DirectTranscription(sys, sys.CreateDefaultContext(), N)
    prog = dirtran.prog()
    prog.AddBoundingBoxConstraint(x0, x0, dirtran.initial_state())
    prog.AddBoundingBoxConstraint(xf, xf, dirtran.final_state())
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] <= 1)
    dirtran.AddConstraintToAllKnotPoints(dirtran.input()[0] >= -1)
    dirtran.AddRunningCost(dirtran.input()[0] ** 2)

    result = Solve(prog)
    assert result.is_success(), "Optimization failed"
    u_sol = dirtran.ReconstructInputTrajectory(result)
    x_sol = dirtran.ReconstructStateTrajectory(result)

    fig, ax = plt.subplots(2, 1)
    x_values = x_sol.vector_values(x_sol.get_segment_times())
    ax[0].plot(x_values[0, :], x_values[1, :], "-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    u_values = u_sol.vector_values(u_sol.get_segment_times())
    ax[1].plot(u_sol.get_segment_times(), u_values.T, "-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


dirtran_example()

One thing that I'm very proud of (it was a lot of work!) is the fact that drake is often smart enough to introspect your system, costs, and constraints and understand whether you have formulated a convex problem or a non-convex one.  The optimization above calls a convex optimization solver.  But if you had passed in a nonlinear system instead, it would have switched to calling a solver that supports nonlinear programming.

## Direct Collocation

With only a minor change, we can use `DirectCollocation` instead of `DirectTranscription`.  This works directly on the continuous-time equations, but even though the system is linear, it gives a nonconvex optimization (make sure you understand why!). Satisfyingly, though, this finds the optimal solution (the true minimum time) without needing a line search!

In [ ]:
def dircol_example():
    # Continuous-time double integrator
    A = np.mat("0 1; 0 0")
    B = np.mat("0; 1")
    C = np.eye(2)
    D = np.zeros((2, 1))
    sys = LinearSystem(A, B, C, D)

    N = 41
    x0 = [-2, 0]
    xf = [0, 0]

    dircol = DirectCollocation(
        system=sys,
        context=sys.CreateDefaultContext(),
        num_time_samples=N,
        minimum_time_step=0.001,
        maximum_time_step=10 / N,
    )
    prog = dircol.prog()
    prog.AddBoundingBoxConstraint(x0, x0, dircol.initial_state())
    prog.AddBoundingBoxConstraint(xf, xf, dircol.final_state())
    dircol.AddConstraintToAllKnotPoints(dircol.input()[0] <= 1)
    dircol.AddConstraintToAllKnotPoints(dircol.input()[0] >= -1)
    dircol.AddEqualTimeIntervalsConstraints()
    dircol.AddFinalCost(dircol.time())

    result = Solve(prog)
    assert result.is_success(), "Optimization failed"
    u_sol = dircol.ReconstructInputTrajectory(result)
    x_sol = dircol.ReconstructStateTrajectory(result)
    print(f"minimum time = {dircol.GetSampleTimes(result)[-1]}")

    fig, ax = plt.subplots(2, 1)
    fig.tight_layout(pad=2.0)
    x_values = x_sol.vector_values(x_sol.get_segment_times())
    ax[0].plot(x_values[0, :], x_values[1, :], ".-")
    ax[0].set_xlabel("q")
    ax[0].set_ylabel("qdot")

    u_values = u_sol.vector_values(u_sol.get_segment_times())
    ax[1].plot(u_sol.get_segment_times(), u_values.T, ".-")
    ax[1].set_xlabel("t")
    ax[1].set_ylabel("u")


dircol_example()

Note that there is a sample in the middle of the solution with u = 0.  Do you understand why?  (Hint: trying changing `N`)

# Minimum-time solutions via convex optimization with Graphs of Convex Sets

Rather than doing a line search to find the minimum time (as we did in the first example above), we can use the powerful machinery of [Graphs of Convex Sets](http://underactuated.mit.edu/optimization.html#gcs) (GCS) to write an optimization problem with the horizon length as (effectly) a decision variable. We accomplish this by forming a serial chain of convex sets, with edge constraints enforcing the (linear) dynamics. But each vertex also has an additional edge that could allow it to transition directly to the target set, if the state is exactly equal to the target.

NOTE: If `options.convex_relaxation=False`, then this program requires a mixed-integer programming solver (like Gurobi).  If `optionst.convex_relaxation=True`, then the resulting optimization is a linear program, and can be solved using open-source solvers.

In [ ]:
def min_time_gcs():
    # Discrete-time approximation of the double integrator.
    dt = 0.05
    A = np.eye(2) + dt * np.mat("0 1; 0 0")
    B = dt * np.mat("0; 1")

    bbox = HPolyhedron.MakeBox([-4, -4], [4, 4])
    gcs = GraphOfConvexSets()
    source = gcs.AddVertex(Point([-2, 0]), "x0")
    target = gcs.AddVertex(Point([0, 0]), "xf")
    prev = source
    # B @ -1 <= x_{next} - A @ x_{prev} <= B @ 1
    dynamics_constraint = LinearConstraint(
        np.concatenate([np.eye(2), -A], axis=1), -B, B
    )

    for n in range(int(3.0 / dt)):
        next = gcs.AddVertex(bbox, f"n={n}")
        e = gcs.AddEdge(prev, next)
        e.AddConstraint(
            Binding[LinearConstraint](
                dynamics_constraint, np.concatenate([next.x(), prev.x()])
            )
        )
        e.AddCost(dt)
        e = gcs.AddEdge(next, target)
        e.AddConstraint(next.x()[0] == target.x()[0])
        e.AddConstraint(next.x()[1] == target.x()[1])
        prev = next

    options = GraphOfConvexSetsOptions()
    options.convex_relaxation = True
    options.max_rounded_paths = 100
    result = gcs.SolveShortestPath(source, target, options)
    assert result.is_success(), "Optimization failed"

    print(f"minimum time = {result.get_optimal_cost()}")

    # display(SVG(pydot.graph_from_dot_data(gcs.GetGraphvizString(result))[0].create_svg()))

    # True solution: min time(q=-2, qdot=0) = 2√2 = 2.828 seconds.
    path = gcs.GetSolutionPath(source, target, result)
    print(f"path length = {(len(path)-1)}")
    x_sol = np.array([result.GetSolution(e.xu()) for e in path])
    np.append(x_sol, [0, 0])

    fig, ax = plt.subplots()
    ax.plot(x_sol[:, 0], x_sol[:, 1], "-")
    ax.set_xlabel("q")
    ax.set_ylabel("qdot")


min_time_gcs()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05031f8c-3586-4b47-be79-7f4893cf2f9d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>